In [7]:
import pandas as pd
from sqlalchemy import create_engine

from elasticsearch import Elasticsearch
import re
from sentence_transformers import SentenceTransformer
import json

import requests

# Consultas de Metadatos: PostgreSQL

In [2]:
usuario = "userPSQL"
contraseña = "passPSQL"
host = "localhost"  
puerto = "5432"
base_datos = "postgres"

engine = create_engine(f"postgresql+psycopg2://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}")

Devolver toda la informmación de todas las asignaturas

In [3]:
query = """SELECT * FROM ASIGNATURAS"""
df = pd.read_sql(query, engine)
df.head()

,id,nombre,numero_creditos,agno_academico,semestre,idioma,titulacion_id
0,615001039,Proyecto de Ciencia de Datos,6,Cuarto curso,Séptimo semestre,Castellano,None
1,615000727,Traductores de Lenguajes de Programacion,6,Tercero curso,Sexto semestre,Castellano,None
2,615000720,Programacion Concurrente y Avanzada,6,Segundo curso,Cuarto semestre,Castellano,None
3,615000335,Sistemas Basados en Computador,6,Cuarto curso,Séptimo semestre,Castellano,None
4,615000222,Taller de Programacion,3,Primer curso,Primer semestre,Castellano,None


Devolver todos los profesores que imparten la asignatura Robótica

In [4]:
query = """
       SELECT P.* 
       FROM PROFESORES P 
       JOIN PROFESORESASIGNATURAS AP ON AP.PROFESOR_ID = P.id
       JOIN ASIGNATURAS A ON A.ID = AP.ASIGNATURA_ID
       WHERE A.NOMBRE = 'Robótica';
    """
df = pd.read_sql(query, engine)
df.head()

,id,nombre,correo_electronico,escuela_id
0,104,Jose Eugenio Naranjo,joseeugenio.naranjo@upm.e\ns,None
1,47,Alfredo Valle Barrio,alfredo.valle@upm.es,None


Devolver los profesores que impartan más de 3 asignaturas

In [5]:
query = """
       SELECT P.NOMBRE, COUNT(A.ID) AS NUM_ASIGNATURAS
       FROM PROFESORES P 
       JOIN PROFESORESASIGNATURAS AP ON AP.PROFESOR_ID = P.ID
       JOIN ASIGNATURAS A ON A.ID = AP.ASIGNATURA_ID
       GROUP BY P.NOMBRE
       HAVING COUNT(A.ID) > 3;
    """
df = pd.read_sql(query, engine)
df.head()

,nombre,num_asignaturas
0,Edgar Talavera Muñoz,5
1,Victor Jose Martinez,6
2,Sergio Alejandro D'antonio,4
3,Joaquin Entrialgo Castaño,5
4,Julio Cesar Hernandez,4


Devolver todas el profesor que imparte el mayor numero de asignaturas de la titulación Grado en Sistemas de Informacion

In [6]:
query = """
        SELECT P.NOMBRE,COUNT(DISTINCT A.ID) AS NUM_ASIGNATURAS
        FROM PROFESORES P
        JOIN PROFESORESASIGNATURAS AP ON AP.PROFESOR_ID = P.ID
        JOIN ASIGNATURAS A ON A.ID = AP.ASIGNATURA_ID
        JOIN TITULACIONESASIGNATURAS TA ON TA.ASIGNATURA_ID = A.ID
        JOIN TITULACIONES T ON T.ID = TA.TITULACION_ID
        WHERE T.NOMBRE = 'Grado en Sistemas de Informacion'
        GROUP BY  P.NOMBRE
        ORDER BY NUM_ASIGNATURAS DESC
        LIMIT 1;
    """
df = pd.read_sql(query, engine)
df.head()

,nombre,num_asignaturas
0,Agustin Yague Panadero,4


# Consultas de contenido: ElasticSearch

In [62]:
directory = "Guias Docentes"
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

es = Elasticsearch("http://localhost:9200"
                  )
index = 'guias_docentes'

## Buscar asignaturas que mencionen un tema en su descripción

In [63]:
query_text = "matemática aplicada"

res = es.search(
    index=index,
    size=5,
    query={
        "match": {
            "descripcion_asignatura": query_text
        }
    }
)

for hit in res.body["hits"]["hits"]:
    print(hit["_source"]["nombre_asignatura"],"(", hit["_source"]["id_asignatura"], ")", "-", hit["_source"]["descripcion_asignatura"], "\n")

Matemática Discreta II ( 615001010 ) - La asignatura Matemática Discreta II complementa la asignatura Matemática Discreta I del primer semestre. En
ella se siguen estudiando algunas de las estructuras discretas importantes en Matemáticas y en Computación,
fundamentalmente Grafos. Se realiza un estudio básico de casi todos los conceptos de Teoría de Grafos,
incidiendo especialmente en los aspectos algorítmicos. También se estudian conceptos generales de Complejidad
de Algoritmos y técnicas de optimización combinatoria en problemas algorítmicos de grafos. 

Matemática Discreta I ( 615001003 ) - Matemática Discreta I es una asignatura en la que se estudian algunas de las estructuras discretas básicas
importantes en Matemáticas y en Computación. Se introduce el concepto de relación sobre un conjunto y se
estudian las diferentes tipos de relaciones y sus propiedades. Estas relaciones son la base para poder desarrollar
posteriormente otras estructuras como la aritmética entera y modular, álg

## Listar todas las asignaturas que mencionen un tema en su descripción

In [64]:
query_text = "seguridad informática"

res = es.search(
    index=index,
    size=250,
    query={
        "match": {"descripcion_asignatura": query_text}
    }
)

vistos = set()

print("Listado de asignaturas cuya descripción contiene", f"'{query_text}'")
for hit in res.body["hits"]["hits"]:
    src = hit["_source"]
    nombre = src["nombre_asignatura"]
    if nombre not in vistos:
        print("-", nombre)
        vistos.add(nombre)


Listado de asignaturas cuya descripción contiene 'seguridad informática'
- Fundamentos de Seguridad
- Seguridad de las Tecnologías de la Información
- Seguridad en Sistemas y Redes
- Seguridad de la Informacion
- Fundamentos Fisicos de la Informatica
- Auditoria y Control Ti
- Probabilidades y Estadística I
- Probabilidad y Estadistica
- Probabilidad y Estadística
- Bases de Datos I
- Inteligencia Artificial
- Matemática Discreta I
- Arquitectura y Diseño Software
- Sistemas de Tiempo Real
- Aspectos Eticos y Sociales
- Logica y Matematica Discreta
- Administracion y Gestion de Bases de Datos
- Bases de Datos Avanzadas
- Bases de Datos Avanzados


## Listar algunas asignaturas que mencionen un tema en sus competencias

In [65]:
query_text = "trabajo en equipo"

res = es.search(
    index=index,
    size=4,
    query={
        "nested": {
            "path": "competencias",
            "query": {
                "match": {
                    "competencias.texto": query_text
                }
            },
            "inner_hits": {}
        }
    }
)

for hit in res.body["hits"]["hits"]:
    print("-", hit["_source"]["nombre_asignatura"])

- Robótica
- Proyecto de Ciencia de Datos
- Fundamentos de Economía y Administración de Empresas
- Seguridad de la Informacion


## Búsqueda de texto libre

In [66]:
query_text = "machine learning avanzado"

res = es.search(
    index=index,
    size=10,
    query={
        "multi_match": {
            "query": query_text,
            "fields": [
                "descripcion_asignatura",
                "competencias.texto",
                "conocimientos_previos"
            ]
        }
    }
)

for hit in res.body["hits"]["hits"]:
    print(hit["_source"]["nombre_asignatura"], "-", hit["_source"]["id_asignatura"], "-", "( score:", hit["_score"], ")")


Aprendizaje Automático I - 615001027 - ( score: 7.377389 )
Probabilidad y Estadistica - 615000239 - ( score: 7.25004 )
Probabilidad y Estadistica - 615000302 - ( score: 7.25004 )
Probabilidad y Estadistica - 615000719 - ( score: 7.25004 )
Probabilidad y Estadística - 615000352 - ( score: 7.25004 )
Fundamentos de Sistemas de Informacion - 615000358 - ( score: 5.4561605 )
Procesamiento Digital de la Señal - 615001057 - ( score: 3.1394656 )
Codificación de la Información - 615001051 - ( score: 3.0039816 )
Bases de Datos Avanzadas - 615000364 - ( score: 2.7928338 )
Bases de Datos Avanzados - 615000256 - ( score: 2.7928338 )


## Búsqueda por keyword código de competencia

In [67]:
vistos = set()
query_text = "CC10"
# Consulta nested
res = es.search(
    index=index,
    size=100,
    query={
        "nested": {
            "path": "competencias",
            "query": {
                "term": {
                    "competencias.codigo": query_text
                }
            }
        }
    },
    _source=["nombre_asignatura"]
)

print(f"Asignaturas que contienen la competencia {query_text}:")
for hit in res.body["hits"]["hits"]:
    nombre = hit["_source"].get("nombre_asignatura", "")
    if nombre and nombre not in vistos:
        print("-", nombre)
        vistos.add(nombre)

Asignaturas que contienen la competencia CC10:
- Taller de Sistemas Operativos
- Sistemas Operativos


## Competencias más comunes en las asignaturas

In [68]:
res = es.search(
    index=index,
    size=0,
    aggs={
        "competencias_mas_comunes": {
            "nested": {"path": "competencias"},
            "aggs": {
                "por_codigo": {
                    "terms": {"field": "competencias.codigo"}
                }
            }
        }
    }
)

for bucket in res.body["aggregations"]["competencias_mas_comunes"]["por_codigo"]["buckets"]:
    print(bucket["key"], "-", bucket["doc_count"])

CT2 - 31
CG01 - 27
CT8 - 22
CB02 - 21
CB01 - 20
CB05 - 20
CE1 - 20
CG06 - 20
CT4 - 20
CT6 - 20


## Búsqueda semántica por campo descripción

In [69]:
def print_subtemas(subtemas, nivel=1, max_nivel=3):
    if nivel > max_nivel:
        return
    indent = "  " * nivel
    for sub in subtemas:
        numero = sub.get("numero", "")
        titulo = sub.get("titulo", "")
        print(f"{indent}{numero} - {titulo}")
        sub_subtemas = sub.get("subtemas", [])
        if sub_subtemas:
            print_subtemas(sub_subtemas, nivel + 1, max_nivel)

In [72]:
query_text = "robots"
query_vector = model.encode(query_text).tolist()

# Búsqueda semántica
res = es.search(
    index=index,
    size=5,
    knn={
        "field": "descripcion_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 100
    },
    _source=["id_asignatura", "nombre_asignatura", "conocimientos_previos", "temario", "descripcion_asignatura"]
)

# Mostrar resultados
for hit in res.body["hits"]["hits"]:
    src = hit["_source"]
    nombre = src.get("nombre_asignatura", "")
    id_asignatura = src.get("id_asignatura", "")
    print(f"{nombre} - {id_asignatura}, (score={hit['_score']:.4f})\n")
    print(f"Conocimientos previos:\n {src.get('conocimientos_previos', '')}\n")

    temario = src.get("temario", [])
    if temario:
        print("Temario:")
        print_subtemas(temario, nivel=1, max_nivel=3)
    print("\n" + "-"*50 + "\n")



Robotica - 615000334, (score=0.7708)

Conocimientos previos:
 - Algebra
- Inteligencia Artificial
- Fundamentos De Programacion
3.2. Otros conocimientos previos recomendados para cursar la asignatura
- Sistemas operativos GNU/Linux
- Lenguaje de programación Python

Temario:
  1 - Introducción a la robótica
    1.1 - Definición de robot y tipos
    1.2 - Desarrollo histórico
    1.3 - Aplicaciones de la robótica
  2 - Percepción y actuación sobre el entorno
    2.1 - Sensores para la navegación
    2.2 - Actuadores específicos para la robótica
    2.3 - Visión artificial
    2.4 - Detección de obstáculos
  3 - Control, planificación y optimización
    3.1 - Control clásico y control borroso
    3.2 - Arquitecturas de control
    3.3 - Políticas de decisión, optimización y aprendizaje por refuerzo
  4 - Robótica software
    4.1 - Arquitecturas de robots software
    4.2 - Programación de softbots
    4.3 - Robotic Process Automation (RPA)

----------------------------------------------

## Búsqueda semántica por conocimientos previos

In [73]:
query_text = "matemáticas básicas y álgebra"
query_vector = model.encode(query_text).tolist()

res = es.search(
    index=index,
    size=3,
    knn={
        "field": "conocimientos_previos_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 100
    },
    _source=["nombre_asignatura", "conocimientos_previos"]
)

for hit in res.body["hits"]["hits"]:
    print(f"{hit['_source']['nombre_asignatura']} (score={hit['_score']:.4f})")
    print(hit["_source"].get("conocimientos_previos", ""), "\n")

Fundamentos Fisicos de la Informatica (score=0.7223)
- Analisis Matematico
3.2. Otros conocimientos previos recomendados para cursar la asignatura
- Conocimientos básicos de física: cinemática, dinámica, trabajo y energía.
- Haber cursado las asignaturas de física y matemáticas del bachillerato científico/tecnológico
- Conocimientos básicos de matemáticas: trigonometría, álgebra vectorial, cálculo diferencial e integral. 

Computación Cuántica (score=0.7098)
- Probabilidad Y Estadistica
- Analisis Matematico
- Fundamentos De Programacion
- Algebra
3.2. Otros conocimientos previos recomendados para cursar la asignatura
- We strongly recommend a good mathematical background (especially in linear algebra) and familiarity with
computational complexity and cryptography. 

Probabilidad y Estadistica (score=0.7097)
- Analisis Matematico
- Logica Y Matematica Discreta
3.2. Otros conocimientos previos recomendados para cursar la asignatura
- Cálculo elemental de probabilidades
- Cálculo diferen

## Búsqueda semántica por competencias

In [74]:
query_text = "trabajo en equipo y comunicación efectiva"
query_vector = model.encode(query_text, normalize_embeddings=True).tolist()

res = es.search(
    index=index,
    size=3,
    knn={
        "field": "competencias_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 50
    },
    _source=["id_asignatura", "nombre_asignatura", "descripcion_asignatura", "competencias.texto", "temario.titulo"]
)

for hit in res.body["hits"]["hits"]:
    print(f"{hit['_source']['nombre_asignatura']} (score={hit['_score']:.4f})\n")
    print("Descripción:", hit["_source"].get("descripcion_asignatura", "")[:300], "\n")
    print("\nCompetencias:")
    for comp in hit["_source"].get("competencias", []):
        print("-", comp["texto"])
    print("\nTemario:")
    index_temario = 0
    for tema in hit["_source"].get("temario", []):
        index_temario = index_temario+1
        print(f"{index_temario}.", tema["titulo"])
    print("\n" + "-"*50 + "\n")


Redes Avanzadas (score=0.6526)

Descripción: La asignatura trabaja los conceptos, técnicas y herramientas necesarias para diseñar, dimensionar, desplegar y
proteger redes orientadas a la provisión de servicios telemáticos. Para ello, se presentarán las arquitecturas de
redes superpuestas empleadas en la actualidad, donde se identificarán cuale 


Competencias:
- CE01 - Capacidad para comprender el entorno de una organización y sus necesidades en el ámbito de las tecnologías de la información y las comunicaciones.
- Capacidad para seleccionar, diseñar, desplegar, integrar y gestionar redes e infraestructuras de comunicaciones en una organización
- Comunicación escrita: Relacionarse eficazmente con otras personas a través de la expresión clara de lo que se piensa, mediante la escritura y los apoyos gráficos.
- Trabajo en equipo: Ser capaz de trabajar como miembro de un equipo interdisciplinar con la finalidad de contribuir a desarrollar proyectos con pragmatismo y sentido de la responsab

## Búsqueda semántica multi-vector

In [76]:
query_text = "aprendizaje profundo y redes neuronales"
query_vector = model.encode(query_text, normalize_embeddings=True).tolist()

res = es.search(
    index=index,
    size=3,
    query={
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": """
                    double score = 0.5 * cosineSimilarity(params.query_vector, 'descripcion_vector') +
                                   0.3 * cosineSimilarity(params.query_vector, 'competencias_vector') +
                                   0.2 * cosineSimilarity(params.query_vector, 'conocimientos_previos_vector');
                    return (score + 1) * 50;  // desplazamos y escalamos
                """,
                "params": {"query_vector": query_vector}
            }
        }
    },
    _source=["nombre_asignatura", "descripcion_asignatura", "competencias.texto", "conocimientos_previos"]
)

for hit in res.body["hits"]["hits"]:
    print(f"{hit['_source']['nombre_asignatura']} (score={hit['_score']:.4f})\n\n")
    print("Descripción:\n", hit["_source"].get("descripcion_asignatura", "")[:600])
    print("\nCompetencias:")
    for comp in hit["_source"].get("competencias", []):
        print("-", comp["texto"])
    print("\nConocimientos previos:", hit["_source"].get("conocimientos_previos", ""), "\n")
    print("\n" + "-"*50 + "\n")


Aprendizaje Automático II (score=58.8683)


Descripción:
 El aprendizaje automático es uno de los campos de estudio en inteligencia artificial que ha obtenido grandes
logros, por ejemplo, en problemas de visión artificial, de reconocimiento del lenguaje natural o de control de robots
autónomos. Dentro del aprendizaje automático, se pueden distinguir diferentes tipos de métodos que son
aplicables a diversas clases de problemas. Esta asignatura se centra principalmente en métodos basados en
redes de neuronas artificiales y en métodos de aprendizaje de modelos probabilísticos.
Dentro del enfoque basado en redes de neuronas artificiales, se presentan mét

Competencias:
- CB01 - Que los estudiantes hayan demostrado poseer y comprender conocimientos en un área de estudio que parte de la base de la educación secundaria general, y se suele encontrar a un nivel que, si bien se apoya en libros de texto avanzados, incluye también algunos aspectos que implican conocimientos procedentes de la vangu

# Consultas de Metaesquema: GraphDB (SPARQL)

In [16]:
GRAPHDB_ENDPOINT = "http://localhost:8000/repositories/asignaturas"

## Consulta 1
Todas las asignaturas impartidas por una escuela

In [17]:
query = """
PREFIX upm: <http://upm.es/ontology/>
SELECT ?nombre_escuela ?nombre_asignatura
WHERE {
    ?s a upm:Escuela .
    ?s upm:nombre ?nombre_escuela .
    ?s upm:imparteTitulacion ?t .
    ?t upm:incluyeAsignatura ?a .
    ?a upm:nombre ?nombre_asignatura .
} LIMIT 10
"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
        nombre_escuela = r.get('nombre_escuela', {}).get('value', '')
        nombre_asignatura = r.get('nombre_asignatura', {}).get('value', '')
        print(f"Escuela: {nombre_escuela}, Asignatura: {nombre_asignatura}")
else:
    print("Error:", response.status_code, response.text)

Escuela: E.T.S De Ing. De Sistemas Informáticos, Asignatura: Taller de Programacion
Escuela: E.T.S De Ing. De Sistemas Informáticos, Asignatura: Arquitectura de Computadores
Escuela: E.T.S De Ing. De Sistemas Informáticos, Asignatura: Algoritmica y Complejidad
Escuela: E.T.S De Ing. De Sistemas Informáticos, Asignatura: Taller de Sistemas Operativos
Escuela: E.T.S De Ing. De Sistemas Informáticos, Asignatura: Seguridad de la Informacion
Escuela: E.T.S De Ing. De Sistemas Informáticos, Asignatura: Fundamentos de Ingenieria del Software
Escuela: E.T.S De Ing. De Sistemas Informáticos, Asignatura: Fundamentos de Economia y Empresa
Escuela: E.T.S De Ing. De Sistemas Informáticos, Asignatura: Fundamentos Fisicos de la Informatica
Escuela: E.T.S De Ing. De Sistemas Informáticos, Asignatura: Robotica
Escuela: E.T.S De Ing. De Sistemas Informáticos, Asignatura: Arquitecturas Avanzadas


## Consulta 2
Encontrar la escuela a la que pertenece el profesor de una asignatura

In [18]:
query = """
PREFIX upm: <http://upm.es/ontology/>
SELECT ?nombre_escuela ?nombre_profesor
WHERE {
    ?e a upm:Escuela .
    ?e upm:nombre ?nombre_escuela .
    ?e upm:imparteTitulacion ?t .
    ?t upm:incluyeAsignatura ?a .
    ?a upm:tieneProfesor ?p .
    ?p upm:nombre ?nombre_profesor .
} LIMIT 10
"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
        nombre_escuela = r.get('nombre_escuela', {}).get('value', '')
        nombre_profesor = r.get('nombre_profesor', {}).get('value', '')
        print(f"Escuela: {nombre_escuela}, Profesor: {nombre_profesor}")
else:
    print("Error:", response.status_code, response.text)

Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Fernando De Mingo Lopez
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Fernando Javier Naharro
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: M. Elvira Martinez De Icaya
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Andres Sevilla De Pablo
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Francisco Aylagas Romero
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Maria Soledad Delgado Sanz
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Javier Huertas Tato
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Cristian Oliver Ramirez
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Antonio Hernando Esteban
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Pilar Manzano Garcia


## Consulta 3
Profesores que imparten más de N asignaturas

In [19]:
query = """
PREFIX upm: <http://upm.es/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?profesor ?nombre (COUNT(?asig) AS ?numAsignaturas) WHERE {
  ?asig rdf:type upm:Asignatura .
  ?asig upm:tieneProfesor ?profesor .
  OPTIONAL { ?profesor upm:nombre ?nombre. }
}
GROUP BY ?profesor ?nombre
HAVING (COUNT(?asig) > 4)
ORDER BY DESC(?numAsignaturas)
LIMIT 10
"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
        nombre_profesor = r.get('nombre', {}).get('value', '')
        num_asignaturas = r.get('numAsignaturas', {}).get('value', '')
        print(f"Profesor: {nombre_profesor}, Número de asignaturas: {num_asignaturas}")
else:
    print("Error:", response.status_code, response.text)

Profesor: Joaquin Gayoso Cabada, Número de asignaturas: 13
Profesor: Cristian Oliver Ramirez, Número de asignaturas: 10
Profesor: Carlos Castilla Ruiz, Número de asignaturas: 9
Profesor: Rafael Miñano Rubio, Número de asignaturas: 9
Profesor: Maria Angeles Mahillo, Número de asignaturas: 9
Profesor: Maria Celia Fernandez Aller, Número de asignaturas: 9
Profesor: Andres Sevilla De Pablo, Número de asignaturas: 8
Profesor: Francisco Aylagas Romero, Número de asignaturas: 8
Profesor: Javier Huertas Tato, Número de asignaturas: 8
Profesor: Jordi Burguet Castell, Número de asignaturas: 8


## Consulta 4
Distribución del número de creditos de cada asignatura por titulación (indicadores: media, mínimo, máximo)

In [ ]:
query = """
PREFIX upm: <http://upm.es/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?titulacion ?titNombre (MIN(?c) AS ?minCreditos) (MAX(?c) AS ?maxCreditos) WHERE {
  ?titulacion a upm:Titulacion .
  OPTIONAL { ?titulacion upm:nombre ?titNombre. }
  ?titulacion upm:incluyeAsignatura ?asig .
  ?asig upm:creditosECTS ?c .
}
GROUP BY ?titulacion ?titNombre
"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
        titNombre = r.get('titNombre', {}).get('value', '')
        minCreditos = r.get('minCreditos', {}).get('value', '')
        maxCreditos = r.get('maxCreditos', {}).get('value', '')
        print(f"Titulación: {titNombre}, Créditos - Mínimo: {minCreditos}, Máximo: {maxCreditos}")
else:
    print("Error:", response.status_code, response.text)

Titulación: Grado en Ingenieria de Computadores, Créditos - Promedio: , Mínimo: 3, Máximo: 9
Titulación: Grado en Ciencia de Datos e Inteligencia Artificial, Créditos - Promedio: , Mínimo: 3, Máximo: 6
Titulación: Grado en Ingenieria del Software, Créditos - Promedio: , Mínimo: 3, Máximo: 9
Titulación: Grado en Sistemas de Informacion, Créditos - Promedio: , Mínimo: 3, Máximo: 9
Titulación: Grado en Tecnologias para la Sociedad de la Informacion, Créditos - Promedio: , Mínimo: 3, Máximo: 9


## Consulta 5
Obtener la dirección postal, web, descripción, director y universidad a la que pertenecen las escuelas.

In [21]:
query = """
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX dbr:  <http://dbpedia.org/resource/>
PREFIX dbpe:  <http://es.dbpedia.org/property/>
PREFIX upm:  <http://upm.es/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>


SELECT ?o ?nombreUni ?l ?w ?abstract ?director
WHERE {
    ?s upm:entidad_dbpedia ?o .

  SERVICE <https://es.dbpedia.org/sparql> {
    ?o dbpe:campus ?u .
    ?u dbpe:nombre ?nombreUni .
    ?o dbpe:localización ?l .
    ?o dbpe:sitioWeb ?w .
    ?o dbpe:director ?director .
    ?o dbo:abstract ?abstract .
  }
}

"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
      o = r.get('o', {}).get('value', '')
      nombreUni = r.get('nombreUni', {}).get('value', '')
      l = r.get('l', {}).get('value', '')
      w = r.get('w', {}).get('value', '')
      abstract = r.get('abstract', {}).get('value', '')
      director = r.get('director', {}).get('value', '')

      print(f"Escuela: {o}")
      print(f"Universidad: {nombreUni}")
      print(f"Localización: {l}")
      print(f"Sitio web: {w}")
      print(f"Director: {director}")
      print(f"Abstract: {abstract}")
      print("-" * 60)
else:
    print("Error:", response.status_code, response.text)

Escuela: http://es.dbpedia.org/resource/Escuela_Técnica_Superior_de_Ingeniería_de_Sistemas_Informáticos_(Universidad_Politécnica_de_Madrid)
Universidad: Universidad Politécnica de Madrid
Localización: Calle Alan Turing s/n , 28031, Madrid,  España
Sitio web: http://www.etsisi.upm.es/
Director: Agustín Yagüe Panadero
Abstract: La Escuela Técnica Superior de Ingeniería de Sistemas Informáticos (ETSISI) es un centro de la Universidad Politécnica de Madrid que imparte las ramas de la Ingeniería Informática, en concreto las titulaciones de Grado en Ingeniería de Software, Ingeniería de Computadores, Sistemas de Información y Tecnologías para la Sociedad de la Información. La Escuela Técnica Superior de Ingeniería de Sistemas Informáticos, junto con la Escuela Técnica Superior de Ingeniería y Sistemas de Telecomunicación, la Escuela Técnica Superior de Ingenieros en Topografía, Geodesia y Cartografía, y el Centro Superior de Diseño de Moda de Madrid, integran el complejo universitario del Ca

## Consulta 6
Obtener información de la universidad

In [22]:
query = """
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX dbr:  <http://dbpedia.org/resource/>
PREFIX dbpe:  <http://es.dbpedia.org/property/>
PREFIX upm:  <http://upm.es/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>


SELECT ?otra
WHERE {
    ?s upm:entidad_dbpedia ?o .

  SERVICE <https://es.dbpedia.org/sparql> {
    ?o dbpe:campus ?u .
    ?u dbpe:nombre ?nombreUni .
    ?u dbo:affiliation ?affiliation .
    ?otra dbo:affiliation ?affiliation .
  }
}

"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
      otra = r.get('otra', {}).get('value', '')
      otra2 = r.get('otra2', {}).get('value', '')

      print(f"Asociada: {otra}")
      print("-" * 60)
else:
    print("Error:", response.status_code, response.text)

Asociada: http://es.dbpedia.org/resource/Universidad_de_Nápoles_Federico_II
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/Universidad_Técnica_de_Delft
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/École_nationale_de_l'aviation_civile
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/Institut_Supérieur_de_l'Aéronautique_et_de_l'Espace
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/Universidad_de_Stuttgart
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/Politécnico_de_Milán
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/Universidad_Politécnica_de_Valencia
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/Universida

## Consulta 7
Asignaturas que imparte el director/a de la escuela.

In [23]:
query = """
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX dbr:  <http://dbpedia.org/resource/>
PREFIX dbpe:  <http://es.dbpedia.org/property/>
PREFIX upm:  <http://upm.es/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>


SELECT ?director_norm ?n
WHERE {
    ?s upm:entidad_dbpedia ?o .
    ?a upm:tieneProfesor ?p .
    ?p upm:nombre ?localName .
    ?a upm:nombre ?n .

  SERVICE <https://es.dbpedia.org/sparql> {
    ?o dbpe:campus ?u .
    ?u dbpe:nombre ?nombreUni .
    ?o dbpe:localización ?l .
    ?o dbpe:sitioWeb ?w .
    ?o dbpe:director ?director .
    ?o dbo:abstract ?abstract .

    BIND(REPLACE(REPLACE(?director, "ü", "u"), "í", "i") AS ?director_norm)

  }

  FILTER(?localName = ?director_norm)

}

"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
      director = r.get('director_norm', {}).get('value', '')
      asignatura = r.get('n', {}).get('value', '')
      print(f"Director: {director}")
      print(f"Asignatura: {asignatura}")
      print("-" * 60)
else:
    print("Error:", response.status_code, response.text)

Director: Agustin Yague Panadero
Asignatura: Fundamentos de Ingenieria de Software
------------------------------------------------------------
Director: Agustin Yague Panadero
Asignatura: Mejores Practicas para Gestion de Servicios
------------------------------------------------------------
Director: Agustin Yague Panadero
Asignatura: Fundamentos de Ingenieria del Software
------------------------------------------------------------
Director: Agustin Yague Panadero
Asignatura: Fundamentos de Ingenieria del Software
------------------------------------------------------------
Director: Agustin Yague Panadero
Asignatura: Fundamentos de Ingenieria del Software
------------------------------------------------------------
Director: Agustin Yague Panadero
Asignatura: Metricas y Modelos para el Control y Gestion de Servicios
------------------------------------------------------------
Director: Agustin Yague Panadero
Asignatura: Desarrollo de Sistemas de Informacion Orientados a
Servicios
-